In [2]:
import pandas as pd
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import torch
import torchvision
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader, random_split
from torchvision.models.densenet import DenseNet121_Weights
import torch.nn as nn
import torch.optim as optim
from PIL import Image, ImageFile
import itertools
from hyperopt import hp, tpe, fmin, Trials, STATUS_OK


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
data = pd.read_csv('/kaggle/input/fracatlas/FracAtlas/dataset.csv')

In [4]:
data.head()

,image_id,hand,leg,hip,shoulder,mixed,hardware,multiscan,fractured,fracture_count,frontal,lateral,oblique
0,IMG0000000.jpg,0,1,0,0,0,0,1,0,0,1,1,0
1,IMG0000001.jpg,0,1,0,0,0,0,1,0,0,1,1,0
2,IMG0000002.jpg,0,1,0,0,0,0,1,0,0,1,1,0
3,IMG0000003.jpg,0,1,0,0,0,0,1,0,0,0,1,1
4,IMG0000004.jpg,0,1,0,0,0,0,1,0,0,0,1,1


In [5]:
base_path = '/kaggle/input/fracatlas/FracAtlas/images'
fractured_path = os.path.join(base_path, 'Fractured')
non_fractured_path = os.path.join(base_path, 'Non_fractured')

In [6]:
image_paths = {img: os.path.join(fractured_path, img) for img in os.listdir(fractured_path)}
image_paths.update({img: os.path.join(non_fractured_path, img) for img in os.listdir(non_fractured_path)})

In [7]:
data_directory = '/kaggle/input/fracatlas/FracAtlas/images'

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [8]:
# This makes PIL ignore bytes beyond the truncation point
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Custom loader that uses PIL and ignores errors in truncated images
def custom_loader(path):
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')
        
dataset = datasets.ImageFolder(root=data_directory, transform=transform, loader=custom_loader)


In [9]:
def set_seed(seed=42):
    torch.manual_seed(seed)
#     np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # for multi-GPU.
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

### Train Validation Test Split

In [10]:
total_size = len(dataset)
train_size = int(0.7 * total_size)
validation_size = int(0.15 * total_size)
test_size = total_size - train_size - validation_size

In [11]:
train_dataset, val_dataset, test_dataset = random_split(
    dataset, [train_size, validation_size, test_size]
)

In [12]:
train_loader = DataLoader(train_dataset, batch_size= 32, shuffle= True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size= 32)

### Transfer Learning

In [13]:
def modify_model(model,dropout_rate,num_classes=2):
    if hasattr(model, 'fc'):
        num_ftrs = model.fc.in_features
        model._fc = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(num_ftrs, num_classes)
        )
    elif hasattr(model, 'classifier'):  # DenseNet
        if isinstance(model.classifier, nn.Sequential):
            *layers, last_layer = model.classifier.children()
            num_ftrs = last_layer.in_features
            new_last_layer = nn.Linear(num_ftrs, num_classes)
            model.classifier = nn.Sequential(*layers, new_last_layer)
        else:
            num_ftrs = model.classifier.in_features
            model.classifier = nn.Linear(num_ftrs, num_classes)
    else:
        raise Exception("Unknown model architecture")
    return model

### Training the Model

In [14]:
from tqdm import tqdm

def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs, train_dataset, val_dataset):
    best_val_accuracy = 0.0
    best_train_accuracy = 0.0
    best_loss = float('inf')

    epoch_bar = tqdm(range(num_epochs), desc="Training", unit="epoch")
    for epoch in epoch_bar:
        model.train()
        total_loss, total_correct = 0, 0

        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            total_correct += (outputs.argmax(1) == labels).sum().item()

        train_accuracy = total_correct / len(train_dataset)

        model.eval()
        val_correct = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                val_correct += (outputs.argmax(1) == labels).sum().item()

        val_accuracy = val_correct / len(val_dataset)
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_train_accuracy = train_accuracy
            best_loss = total_loss / len(train_loader)

        # Update tqdm description for epochs
        epoch_bar.set_description(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {total_loss/len(train_loader):.4f}, Train Acc: {train_accuracy:.4f}, Val Acc: {val_accuracy:.4f}")

    return model, best_val_accuracy, best_train_accuracy, best_loss


### Hyperparameter Tuning

In [15]:
import csv
import os

def log_results(params, val_accuracy,train_accuracy, train_loss, file_path='hyperparameters_log.csv'):
    # Check if the file exists. If not, create it and write the header
    file_exists = os.path.isfile(file_path)

    with open(file_path, 'a', newline='') as csvfile:
        fieldnames = ['lr', 'batch_size', 'epochs', 'optimizer', 'weight_decay', 'dropout_rate', 'train_accuracy', 'val_accuracy', 'train_loss']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        if not file_exists:
            writer.writeheader()  # Write header only once

        # Write data
        writer.writerow({
            'lr': params['lr'],
            'batch_size': params['batch_size'],
            'epochs': params['epochs'],
            'optimizer': params['optimizer'],
            'weight_decay': params['weight_decay'],
            'dropout_rate': params['dropout_rate'],
            'train_accuracy': train_accuracy,
            'val_accuracy': val_accuracy,
            'train_loss': train_loss
        })


In [16]:
densenet_model = models.densenet121(weights=DenseNet121_Weights.IMAGENET1K_V1)

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 163MB/s] 


In [17]:
def objective(params):
    # Unpack hyperparameters
    lr = params['lr']
    batch_size = int(params['batch_size'])
    num_epochs = int(params['epochs'])
    optimizer_name = params['optimizer']
    wd = params['weight_decay']
    dropout_rate = params['dropout_rate']

    # Data Loading
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Model Loading and Modification
    model = modify_model(densenet_model, dropout_rate)

    # Setup optimizer
    optimizer_class = getattr(optim, optimizer_name)
    optimizer = optimizer_class(model.parameters(), lr=lr, weight_decay=wd)
    criterion = nn.CrossEntropyLoss()

    # Train the model
    trained_model, best_val_accuracy, best_train_accuracy, best_loss = train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs, train_dataset, val_dataset)

    # Logging results (optional)
    log_results(params, best_val_accuracy, best_train_accuracy, best_loss)

    # Return metric to minimize
    return {'loss': -best_val_accuracy, 'status': STATUS_OK, 'model': trained_model}


In [18]:
space = {
    'lr': hp.loguniform('lr', np.log(0.0001), np.log(0.01)),
    'batch_size': hp.choice('batch_size', [32, 64]),
    'epochs': hp.choice('epochs', [2, 3]),
    'optimizer': hp.choice('optimizer', ['Adam', 'SGD']),
    'weight_decay': hp.loguniform('weight_decay', np.log(0.0001), np.log(0.01)),
    'dropout_rate': hp.choice('dropout_rate', [0.1, 0.3, 0.5])
}

In [19]:
trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=15,  # Number of iterations
            trials=trials)

print("Best hyperparameters:", best)


  0%|          | 0/15 [00:00<?, ?trial/s, best loss=?]

Training:   0%|          | 0/2 [00:00<?, ?epoch/s]

Epoch 1/2, Train Loss: 0.5081, Train Acc: 0.7869, Val Acc: 0.8366:   0%|          | 0/2 [11:23<?, ?epoch/s]

Epoch 1/2, Train Loss: 0.5081, Train Acc: 0.7869, Val Acc: 0.8366:  50%|#####     | 1/2 [11:23<11:23, 683.50s/epoch]

Epoch 2/2, Train Loss: 0.4412, Train Acc: 0.8202, Val Acc: 0.8366:  50%|#####     | 1/2 [22:10<11:23, 683.50s/epoch]

Epoch 2/2, Train Loss: 0.4412, Train Acc: 0.8202, Val Acc: 0.8366: 100%|##########| 2/2 [22:10<00:00, 661.79s/epoch]

Epoch 2/2, Train Loss: 0.4412, Train Acc: 0.8202, Val Acc: 0.8366: 100%|##########| 2/2 [22:10<00:00, 665.04s/epoch]


  7%|▋         | 1/15 [22:10<5:10:21, 1330.10s/trial, best loss: -0.8366013071895425]

Training:   0%|          | 0/3 [00:00<?, ?epoch/s]

Epoch 1/3, Train Loss: 0.5689, Train Acc: 0.8076, Val Acc: 0.8350:   0%|          | 0/3 [09:02<?, ?epoch/s]

Epoch 1/3, Train Loss: 0.5689, Train Acc: 0.8076, Val Acc: 0.8350:  33%|###3      | 1/3 [09:02<18:04, 542.50s/epoch]

Epoch 2/3, Train Loss: 0.4600, Train Acc: 0.8226, Val Acc: 0.8350:  33%|###3      | 1/3 [18:04<18:04, 542.50s/epoch]

Epoch 2/3, Train Loss: 0.4600, Train Acc: 0.8226, Val Acc: 0.8350:  67%|######6   | 2/3 [18:04<09:02, 542.45s/epoch]

Epoch 3/3, Train Loss: 0.4572, Train Acc: 0.8184, Val Acc: 0.8350:  67%|######6   | 2/3 [27:09<09:02, 542.45s/epoch]

Epoch 3/3, Train Loss: 0.4572, Train Acc: 0.8184, Val Acc: 0.8350: 100%|##########| 3/3 [27:09<00:00, 543.54s/epoch]

Epoch 3/3, Train Loss: 0.4572, Train Acc: 0.8184, Val Acc: 0.8350: 100%|##########| 3/3 [27:09<00:00, 543.25s/epoch]


 13%|█▎        | 2/15 [49:19<5:26:22, 1506.38s/trial, best loss: -0.8366013071895425]

...

 87%|████████▋ | 13/15 [5:45:11<57:13, 1716.97s/trial, best loss: -0.8611111111111112]  

Training:   0%|          | 0/3 [00:00<?, ?epoch/s]

Epoch 1/3, Train Loss: 0.6587, Train Acc: 0.8184, Val Acc: 0.8350:   0%|          | 0/3 [14:39<?, ?epoch/s]

Epoch 1/3, Train Loss: 0.6587, Train Acc: 0.8184, Val Acc: 0.8350:  33%|###3      | 1/3 [14:39<29:19, 879.87s/epoch]

Epoch 2/3, Train Loss: 0.6034, Train Acc: 0.8198, Val Acc: 0.8350:  33%|###3      | 1/3 [29:02<29:19, 879.87s/epoch]

Epoch 2/3, Train Loss: 0.6034, Train Acc: 0.8198, Val Acc: 0.8350:  67%|######6   | 2/3 [29:02<14:29, 869.70s/epoch]

Epoch 3/3, Train Loss: 0.5607, Train Acc: 0.8198, Val Acc: 0.8350:  67%|######6   | 2/3 [42:54<14:29, 869.70s/epoch]

Epoch 3/3, Train Loss: 0.5607, Train Acc: 0.8198, Val Acc: 0.8350: 100%|##########| 3/3 [42:54<00:00, 852.52s/epoch]

Epoch 3/3, Train Loss: 0.5607, Train Acc: 0.8198, Val Acc: 0.8350: 100%|##########| 3/3 [42:54<00:00, 858.17s/epoch]


 93%|█████████▎| 14/15 [6:28:05<32:55, 1976.00s/trial, best loss: -0.8611111111111112]

Training:   0%|          | 0/2 [00:00<?, ?epoch/s]

Epoch 1/2, Train Loss: 0.6798, Train Acc: 0.7817, Val Acc: 0.8497:   0%|          | 0/2 [18:31<?, ?epoch/s]

Epoch 1/2, Train Loss: 0.6798, Train Acc: 0.7817, Val Acc: 0.8497:  50%|#####     | 1/2 [18:31<18:31, 1111.02s/epoch]

Epoch 2/2, Train Loss: 0.6649, Train Acc: 0.8446, Val Acc: 0.8513:  50%|#####     | 1/2 [37:26<18:31, 1111.02s/epoch]

Epoch 2/2, Train Loss: 0.6649, Train Acc: 0.8446, Val Acc: 0.8513: 100%|##########| 2/2 [37:26<00:00, 1125.41s/epoch]

Epoch 2/2, Train Loss: 0.6649, Train Acc: 0.8446, Val Acc: 0.8513: 100%|##########| 2/2 [37:26<00:00, 1123.25s/epoch]


100%|██████████| 15/15 [7:05:32<00:00, 1702.14s/trial, best loss: -0.8611111111111112]
Best hyperparameters: {'batch_size': 0, 'dropout_rate': 2, 'epochs': 0, 'lr': 0.00019763053882814122, 'optimizer': 0, 'weight_decay': 0.00045134920151192344}


In [22]:
model_save_dir = "/kaggle/working"

In [23]:
best_model = trials.results[np.argmin([r['loss'] for r in trials.results])]['model']
torch.save(best_model.state_dict(), os.path.join(model_save_dir, "best_model.pth"))


## Testing on Test dataset

In [30]:
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

#### Loading the model

In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [32]:
# Define the model architecture (should be same as the one used for training)
model = models.densenet121(pretrained=False)
num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, 2)  # num_classes should be set as before

# Load the saved model weights
model.load_state_dict(torch.load('best_model.pth'))
model = model.to(device)


In [33]:
def test_model(model, test_loader, criterion, device):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    total_loss = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    average_loss = total_loss / len(test_loader)
    return accuracy, average_loss


In [34]:
criterion = nn.CrossEntropyLoss()  # Assuming you are using CrossEntropyLoss
test_accuracy, test_loss = test_model(model, test_loader, criterion, device)

print(f'Test Accuracy: {test_accuracy:.2f}%, Test Loss: {test_loss:.4f}')


Test Accuracy: 85.15%, Test Loss: 0.6564
